In [3]:
#주제 식별
!pip install gensim
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from gensim import corpora, models
import nltk
import feedparser

C:\Users\ghd92\anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [8]:
class IdentifyingTopicExample:  
    
    def getDocuments(self):      
        #URL의 문서를 다운로드하고 feed라는 변수에 저장
        url = 'https://sports.yahoo.com/mlb/rss.xml'
        feed = feedparser.parse(url)
        
        self.documents = []
        for entry in feed['entries'][:5]: #상위 5개의 entries중 
            text = entry['summary'] # sunnary 문서를 text로 저장
            if 'ex' in text:
                continue
            self.documents.append(text)
            print("-- {}".format(text))
            
        print("\n")
        print("INFO: Fetching documents from {} completed".format(url))
        print("\n")

    def cleanDocuments(self):
        tokenizer = RegexpTokenizer(r'[a-zA-Z]+') # 각 토큰은 a-z, A-Z사이의 문자로 구성
        en_stop = set(stopwords.words('english'))
        self.cleaned = []
        
        for doc in self.documents:
            lowercase_doc = doc.lower()
            words = tokenizer.tokenize(lowercase_doc) # NLTK의 지정된 불용어를 제거
            non_stopped_words = [i for i in words if not i in en_stop]
            self.cleaned.append(non_stopped_words)
            
        print("INFO: Clearning {} documents completed".format(len(self.documents)))
        print("\n")

    def doLDA(self):
        dictionary = corpora.Dictionary(self.cleaned)
        corpus = [dictionary.doc2bow(cleandoc) for cleandoc in self.cleaned]
        #토픽 수를 2로 정한 말뭉치 모델을 생성하고 어휘크기를 설정
        ldamodel = models.ldamodel.LdaModel(corpus, num_topics=2, id2word = dictionary)
        print(ldamodel.print_topics(num_topics=2, num_words=4)) # 토픽당 4개의 단어 출력
        print("\n")

    def run(self): #함수 순서대로 실행
        self.getDocuments()
        self.cleanDocuments()
        self.doLDA()

In [9]:
if __name__ == '__main__':
    topicExample = IdentifyingTopicExample()
    topicExample.run()


-- Former Dodgers star Gil Hodges and Negro Leagues legend Buck O'Neil headline a big Hall of Fame class added via its committee voting.
-- Ron Darling hesitated to make the Max Scherzer - Tom Brady comparison, knowing it might sound like overstatement. But it seemed appropriate to him.
-- Will Seiya Suzuki land in the American League East? The Red Sox, Yankees and Blue Jays are all aggressively pursuing the Japanese star, per a report.
-- Today’s guest columnist is Andrew Zimbalist, professor of economics at Smith College and author of several books on the economics of baseball. He has consulted in the past both for the MLB Players Association and the commissioner’s office. MLB has entered its first work stoppage since 1995. It is not surprising to see the union […]
-- Gil Hodges and Buck O'Neil were among the six elected to the National Baseball Hall of Fame following Sunday's committee meetings.


INFO: Fetching documents from https://sports.yahoo.com/mlb/rss.xml completed


INFO: C